In [295]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [296]:
# settings
np.random.seed(2020)
name = "Dblp"
d = 128
come_model_type = "BGMM"
ks = [2, 5, 10, 20]

In [297]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [ ]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


k = 2


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.447244,-0.590060,-0.327605,0.759601,0.407560,0.058691,0.100031,0.519430,0.105071,0.309200,...,0.026815,-0.170982,-0.574228,-0.198281,-0.451050,0.011072,-0.094810,-0.301875,1,0
1,0.295918,-0.558974,-0.200630,0.670498,0.255454,0.012444,0.046877,0.438116,-0.062488,0.382698,...,-0.077961,-0.241738,-0.520454,-0.098120,-0.499365,-0.048436,0.035603,-0.250949,1,2
2,0.533605,-0.535543,-0.390154,0.778972,0.445067,0.030342,0.050596,0.546266,0.181827,0.356746,...,0.023890,-0.090602,-0.596374,-0.249288,-0.443732,-0.064824,-0.046458,-0.358123,1,2
3,0.422736,-0.478823,-0.391336,0.637420,0.339128,-0.047656,0.112250,0.464919,0.160469,0.317239,...,-0.017747,-0.144513,-0.465714,-0.285970,-0.305572,0.000968,-0.058966,-0.357066,1,0
4,0.325676,-0.428440,-0.432361,0.703664,0.361787,0.000723,-0.070736,0.472260,0.272814,0.312631,...,0.039992,-0.145147,-0.551694,-0.296413,-0.290567,-0.063562,-0.169452,-0.348867,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.257980,-0.395774,-0.185478,0.552952,0.410871,-0.034553,-0.067511,0.413107,0.183474,0.325979,...,-0.171239,-0.060339,-0.347124,-0.287109,-0.128301,0.030241,0.094815,-0.255052,1,0
13180,-0.020099,-0.540597,-0.269626,0.003425,0.431760,0.091559,0.119777,0.100602,0.358634,0.445394,...,-0.318657,0.178803,-0.034317,0.113252,0.093031,-0.522664,0.109797,0.340559,0,3
13181,-0.118595,-0.484969,-0.095687,0.009911,0.409799,-0.006928,0.062702,0.008954,0.505634,0.593392,...,-0.070694,0.259097,-0.084647,0.026951,-0.060800,-0.500006,0.138947,0.363550,0,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.070362,-0.349749,-0.039339,0.317448,0.314261,-0.113492,0.198119,0.249685,0.185212,0.415116,...,-0.116847,0.216113,-0.288452,-0.003144,-0.033984,-0.015064,0.099052,0.109553,0.386074,2.190382
std,0.158703,0.187695,0.180287,0.210781,0.165113,0.194175,0.185949,0.172968,0.186557,0.152076,...,0.149657,0.157409,0.220734,0.148219,0.179455,0.163438,0.173225,0.249231,0.486866,1.315339
min,-0.508291,-1.023050,-0.624764,-0.434922,-0.292775,-0.728536,-0.466597,-0.464700,-0.451996,-0.187440,...,-0.741044,-0.457601,-0.949299,-0.708024,-0.760009,-0.728847,-0.512301,-0.561539,0.000000,0.000000
25%,-0.032193,-0.487592,-0.161421,0.168124,0.208116,-0.263246,0.068560,0.132015,0.043462,0.317043,...,-0.216404,0.111103,-0.451154,-0.091365,-0.161933,-0.116730,-0.018078,-0.096275,0.000000,1.000000
50%,0.071219,-0.348648,-0.047358,0.329033,0.321037,-0.114689,0.197320,0.254364,0.171024,0.419377,...,-0.114634,0.219112,-0.293272,-0.000578,-0.035395,-0.010756,0.092400,0.131962,0.000000,3.000000
75%,0.174777,-0.212791,0.073073,0.479097,0.422816,0.028225,0.326423,0.364569,0.318899,0.515942,...,-0.016848,0.319095,-0.113045,0.089160,0.090639,0.094135,0.211754,0.305395,1.000000,3.000000
max,0.637020,0.296944,0.650146,0.993297,0.928177,0.465188,0.867931,0.845109,0.869033,0.919915,...,0.413959,0.770119,0.419585,0.654550,0.548400,0.575931,0.699623,0.888070,1.000000,4.000000


# Community Detection

In [ ]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

In [ ]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

# Node Classification

In [ ]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0,128) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.2f}")
    print(f"  macro_f1: {macro_f1*100:.2f}")